In [11]:
import pandas as pd
from dataclasses import dataclass
from typing import Optional

@dataclass
class RunContext:
    run_id: str = None
    started_at: str = None
    data_path: str = None
    target: str = None
    output_dir: str = None
    seed: int = None
    test_size: float = None
    max_replans: int = None

In [12]:
ctx = RunContext()

def infer_target_column(df: pd.DataFrame) -> Optional[str]:
    """
    Heuristic target inference:
      - prefer common target-like column names
      - else last column if it has relatively low cardinality
    """
    candidates = ["target", "label", "class", "y", "outcome"]
    lower_map = {c.lower(): c for c in df.columns}
    for k in candidates:
        if k in lower_map:
            return lower_map[k]

    last = df.columns[-1]
    uniq = df[last].nunique(dropna=True)
    n = len(df)
    if n > 0 and (uniq <= 50 or (uniq / max(n, 1) < 0.05)):
        return last
    return None

In [13]:
# Input path where shown to load dataset
path = "./data/demo.csv"
target = None # <- input target
print(f"Loading dataset: {path}")
df = pd.read_csv(path)
print(f"Loaded {df.shape[0]} rows × {df.shape[1]} cols")


Loading dataset: ./data/demo.csv
Loaded 20 rows × 6 cols


In [14]:
inferred = infer_target_column(df)
if not inferred:
    raise ValueError("Could not infer target column. Please provide --target <name>.")
# Update context with inferred target name
ctx.target = inferred
print(f"Inferred target: {inferred}")


Inferred target: label
